In [ ]:
import ssl
import paho.mqtt.client as mqtt
import time

from config import CONFIG
# config.py is not synced because it contains credentials to the MQTT broker, see config_sample.py for an example.

CONFIG

In [ ]:
# Define the MQTT callbacks
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to HiveMQ Cloud")
        client.subscribe(CONFIG.MQTT_TOPIC)
    else:
        print("Connection failed, return code:", rc)

def on_message(client, userdata, msg):
    print(f"[{msg.topic}] {msg.payload.decode('utf-8')}")
    # Append the message with timestamp to a txt file

    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    record = f"{timestamp}, {msg.payload.decode('utf-8')}\n"
    with open("messages.txt", "a") as f:
        f.write(record)

# Setup the MQTT client
client = mqtt.Client()

client.username_pw_set(CONFIG.MQTT_USER, CONFIG.MQTT_PASSWORD)

client.tls_set(
    cert_reqs=ssl.CERT_REQUIRED,
    tls_version=ssl.PROTOCOL_TLS,
)

# Add the MQTT callbacks
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker and start the loop
client.connect(CONFIG.MQTT_BROKER, CONFIG.MQTT_PORT, keepalive=60)
client.loop_forever()